Este conjunto de dados contém os recursos finais de tratamento de dados.
Todos os valores faltantes foram tratados e categorias semânticas foram criadas (ex. FIRST_TIME_CLIENT).

In [24]:
import pandas as pd
import seaborn as sns

In [25]:
df_hair_salon = pd.read_csv('../data/raw/hair_salon_no_show_wrangled_df.csv')
df_hair_salon.head()

,Unnamed: 0,book_tod,book_dow,book_category,book_staff,last_category,last_staff,last_day_services,last_receipt_tot,last_dow,...,last_prod_flag,last_cumrev,last_cumbook,last_cumstyle,last_cumcolor,last_cumprod,last_cumcancel,last_cumnoshow,noshow,recency
0,0,afternoon,Wednesday,STYLE,JJ,NaN,NaN,0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1,morning,Wednesday,COLOR,JOANNE,NaN,NaN,0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2,NaN,Wednesday,STYLE,JJ,COLOR,JOANNE,2,262.0,Wednesday,...,1,262,2,1,1,2,0,0,0,35
3,3,afternoon,Thursday,STYLE,KELLY,NaN,NaN,0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,evening,Saturday,STYLE,KELLY,STYLE,KELLY,1,35.0,Thursday,...,0,35,1,1,0,0,0,0,0,9


In [26]:
df_hair_salon.columns

Index(['Unnamed: 0', 'book_tod', 'book_dow', 'book_category', 'book_staff',
       'last_category', 'last_staff', 'last_day_services', 'last_receipt_tot',
       'last_dow', 'last_tod', 'last_noshow', 'last_prod_flag', 'last_cumrev',
       'last_cumbook', 'last_cumstyle', 'last_cumcolor', 'last_cumprod',
       'last_cumcancel', 'last_cumnoshow', 'noshow', 'recency'],
      dtype='object')

In [27]:
df_hair_salon['receipt_per_service'] = df_hair_salon['last_receipt_tot'] / (df_hair_salon['last_day_services'] + 0.1)

In [28]:
df_hair_salon['noshow_rate'] = df_hair_salon['last_cumnoshow'] / (df_hair_salon['last_cumbook'] + 0.1)

In [29]:
selected_features = [
    'book_tod',
    'book_dow',
    'recency',

    'last_noshow',
    'last_cumnoshow',
    'last_cumbook',
    'last_cumcancel',

    'receipt_per_service',
    'noshow_rate',

    'book_category',
    'last_category',
    'noshow'
]
df_final = df_hair_salon[selected_features].copy()




In [30]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1952 entries, 0 to 1951
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   book_tod             1717 non-null   object 
 1   book_dow             1952 non-null   object 
 2   recency              1952 non-null   int64  
 3   last_noshow          1952 non-null   int64  
 4   last_cumnoshow       1952 non-null   int64  
 5   last_cumbook         1952 non-null   int64  
 6   last_cumcancel       1952 non-null   int64  
 7   receipt_per_service  1952 non-null   float64
 8   noshow_rate          1952 non-null   float64
 9   book_category        1952 non-null   object 
 10  last_category        1003 non-null   object 
 11  noshow               1952 non-null   int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 183.1+ KB


In [31]:
df_final.isna().mean().sort_values(ascending=False)


last_category          0.486168
book_tod               0.120389
recency                0.000000
book_dow               0.000000
last_noshow            0.000000
last_cumnoshow         0.000000
last_cumcancel         0.000000
last_cumbook           0.000000
receipt_per_service    0.000000
noshow_rate            0.000000
book_category          0.000000
noshow                 0.000000
dtype: float64

A ausência de registros na variável last_category indica que o cliente não possuía histórico anterior no sistema, sendo interpretado como um cliente de primeira visita (first-time client). Já os valores ausentes na variável book_tod são atribuídos a inconsistências ou falhas no processo de registro dos dados.

In [32]:
df_final['last_category'] = df_final['last_category'].fillna('FIRST_TIME_CLIENT')
df_final['book_tod'] = df_final['book_tod'].fillna('Unknown')

In [33]:
df_final.describe()

,recency,last_noshow,last_cumnoshow,last_cumbook,last_cumcancel,receipt_per_service,noshow_rate,noshow
count,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000
mean,14.402152,0.076332,0.422643,1.906762,0.280225,27.006671,0.064216,0.114754
std,21.079495,0.265597,1.319786,3.315854,0.853646,43.374978,0.178585,0.318807
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,27.000000,0.000000,0.000000,2.000000,0.000000,50.000000,0.000000,0.000000
max,133.000000,1.000000,9.000000,20.000000,8.000000,318.181818,0.985915,1.000000


In [34]:
df_final[['book_category','last_category','book_tod','book_dow']].nunique()


book_category    3
last_category    4
book_tod         4
book_dow         7
dtype: int64

In [35]:
df_final.to_csv(
    '../data/processed/hair_salon_model_dataset.csv',
    index=False
)